In [ ]:
#importing diffrent libraries that are necessary
#import the library nltk responsible for different natural language tools
import nltk
#import regular rexpression library
import re
#import math library to use sqrt later
from math import *
#import the rooting library
from nltk.stem.porter import PorterStemmer
#import stopwords from nltk
from nltk.corpus import stopwords


#create instance of the class porterStemmer
stemmer = PorterStemmer()

#downloads stopwords initially in english
nltk.download('stopwords')
#change stopwords to french
stop=stopwords.words('french')

#list of source docs
source=["doc1.txt","doc2.txt","doc3.txt","doc4.txt","doc5.txt","doc6.txt","doc7.txt","doc8.txt","doc9.txt","doc10.txt"]

#indexation function
def liste_Stem(contenu,stop):
  #create a list of 'mots' containing the content 'contenu' in lowercase
  mots=re.split(r"\W", contenu.lower())
  #create list v  containing the words of mots  that differ from stopwords
  v=[t for t in mots if t not in stop]
  #create list filtered_words which contains the words of v that dosen't match date format
  filtered_words = [t for t in v if not re.match(r"\d+", t)]
  #create empty list 'mots_final'
  mots_final=[]
  #loop the filtered_words
  for c in filtered_words:
      #root 'raciniser" every word of  filtered_words
      x=stemmer.stem(c)
      #create list of rooted words
      mots_final.append(x)
  return mots

#test stemming
#mots=liste_Stem(contenu,stop)
#print(mots)

#fonction calcul local de tf
def calcul_TF(contenu):
  #create empty dict
   d={}
   # the total counts of words is the length of contenu
   total = len(contenu)
   #create empty dict for term frequency
   tf={}
   #iterate on each word in the content
   for word in contenu:
    # if the word is not in the dict
    if word not in d:
      #word hzs value 0
      d[word]=0
    #incriment word value
    d[word] += 1
    #term freq is frequency of term in doc / total counts of all words in the doc
    tf[word] = d[word] / total
   return tf

#tf=calcul_TF(mots)
#print("tf est",tf)


#fonction calcul global tf
def TF_global(source):
    #create empty dict
    d = {}
    # the total counts of words is intially 0
    total_words = 0
    #iterate on all documents in the provided source
    for fich in source:
        #open the content of each doc to read it
        contenu = open(fich, "r").read()
        #stem the content
        mots = liste_Stem(contenu,stop)
        #increment count of total word by the length of each doc content
        total_words += len(mots)
        # call for term freq function
        tf = calcul_TF(mots)
        #ierate on each word on term freq
        for word in tf:
             # if the word is not in the dict
            if word not in d:
                #word count is set to 0
                d[word] = 0
            #increment term freq of word
            d[word] += tf[word]
    # Normalisation TF by division tf ib total number of words
    for word in d:
        d[word] /= total_words

    return d

#d2 = TF_global(source)
#print("Global TF:", d2)

#function to calculate weight of request
def calculate_request_weight(indexed_req):
    #create empty dict
    request_weight = {}
    #total words is length of request
    total_words = len(indexed_req)
    # Calculate term frequency (TF) for each token in the query
    for token in indexed_req:
        if token not in request_weight:
            #TF token is set to 0
            request_weight[token] = 0
        request_weight[token] += 1 / total_words  # TF for the request is uniform for each token

    return request_weight

#function to calculate weight of doc
def calcul_weight_doc(source):
    #create empty dict
    d = {}
    #total docs is length of request
    total_docs = len(source)
    #iterate on fich in the privided source
    for fich in source:
        #open the content of each doc to read it
        contenu = open(fich, "r").read()
        #root the content of each doc while getting rid of stopwords
        stems = liste_Stem(contenu,stop)
        # call for term freq function
        tf = calcul_TF(stems)
        #iterate on stem,weight in dict of tf values
        for stem, weight in tf.items():
            if stem not in d:
                #create empty dict for each unfound stem
                d[stem] = {}
            #affect weight for each doc if each stem
            d[stem][fich] = weight
    return d

#d=calcul_weight_doc(source)
#print (d)


#function to create inversed doc
def create_index_inv(source):
  #calculate weight of provided source
  w=calcul_weight_doc(source)
  #open new file to write result in
  with open(r"C:\Users\TASNIME\Documents\indx mult\fich-inv.txt", "w") as file:
    file.write("stem----doc(i)-----W(i).......\n")
    #iterate on each doc in source
    for fich in source:
      #open the content of each doc to read it
      contenu=open(fich, "r").read()
      #root the content of each doc while getting rid of stopwords
      stems = liste_Stem(contenu,stop)
      #iterate on each stem
      for s in stems:
        #write on the file the stem,doc and weight of each stem
        file.write(s+"----"+fich+"----"+str(w[s])+"\n")
  print("doc a été crér avec succées")


#function of user request
def saisie():
  req=input("please enter your request: ")
  return req

#function to calculate similarity using cosinus function
def cosinus_calculate_similarity(weights_request, documents_weight):
    #create empty doc for similarity
    similarity_scores = {}
    # iterate on request words and weight
    for req_word, req_weight in weights_request.items(): #weigts_request.items is the weight of request
        # if the request word exists in document weight dict
        if req_word in documents_weight:
            # iterate on documrents and weight
            for doc, weight_doc in documents_weight[req_word].items():# documents_weight[req_word].items() is the weight of doc
                #intialize key with a default similarity score of 0 to make sure each key has a value
                similarity_scores.setdefault(doc, 0)
                #give each doc a similarity swore using the cosinus equation
                similarity_scores[doc] += weight_doc * req_weight / (sqrt(weight_doc) * sqrt(req_weight))
    return similarity_scores


# dice similarity function
def dice_calculate_similarity(weights_request, documents_weight):
   similarity_scores = {}
   for req_word, req_weight in weights_request.items():
        if req_word in documents_weight:
            for doc, weight_doc in documents_weight[req_word].items():
                similarity_scores.setdefault(doc, 0)
                similarity_scores[doc] += 2 * (weight_doc * req_weight) / weight_doc + req_weight
   return similarity_scores


#jaccard similarity function
def jaccard_calculate_similarity(weights_request, documents_weight):
   similarity_scores = {}
   for req_word, req_weight in weights_request.items():
        if req_word in documents_weight:
            for doc, weight_doc in documents_weight[req_word].items():
                similarity_scores.setdefault(doc, 0)
                similarity_scores[doc] += weight_doc * req_weight / (weight_doc + req_weight) - (weight_doc * req_weight)
   return similarity_scores


#function to  return a given number of relvent doc
def relevant_documents(x,similarity_scores):
    #sort documents based on the similarity score,decendant
    sorted_documents = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_documents[:x]

#create inversed document
create_index_inv(source)

#get the user's request
req=saisie()

#indexed request
indx_req=liste_Stem(req,stop)
print("\n indexed request: " ,indx_req)
#calculate the document weight
documents_weight = calcul_weight_doc(source)
#calculate the request weight
req_weight=calculate_request_weight(indx_req)
print("\n request weight: ",req_weight)

# Calculate similarity between request and documents
similarity_scores = cosinus_calculate_similarity(req_weight, documents_weight)
print("\n similarity_score: ",similarity_scores)

#print the relevant documents
x=int(input("\n enter number of relevant ocs to show: "))
relv_doc=relevant_documents(x,similarity_scores)
print(relv_doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


doc a été crér avec succées
please enter your request: homme soleil

 indexed request:  ['homme', 'soleil']

 request weight:  {'homme': 0.5, 'soleil': 0.5}

 similarity_score:  {'doc1.txt': 0.04173919355648411, 'doc5.txt': 0.03990434422338111, 'doc7.txt': 0.06835859270246632, 'doc9.txt': 0.042796049251091296}

 enter number of relevant ocs to show: 3
[('doc7.txt', 0.06835859270246632), ('doc9.txt', 0.042796049251091296), ('doc1.txt', 0.04173919355648411)]


# New Section